In [1]:
import csv
import os
import glob
import re
from pandas import DataFrame, Series
from openslide import open_slide
from PIL import Image
import timeit
import time
import math

In [2]:
train_paths = ["/scratch/kk4ze/data_svs/valid/Celiac", "/scratch/kk4ze/data_svs/valid/EE", "/scratch/kk4ze/data_svs/valid/Normal", 
              "/scratch/kk4ze/data_svs/train/Celiac", "/scratch/kk4ze/data_svs/train/EE", "/scratch/kk4ze/data_svs/train/Normal"]
#train_paths = ["/scratch/kk4ze/data_svs/train/EE"]


images = {}
images_by_folder = {}
for train_path in train_paths:
    images_by_folder[str(train_path)] = []
    files = glob.glob(os.path.join(train_path, '*.svs'))
    for fl in files:
        flbase = os.path.basename(fl)
        flbase_noext = os.path.splitext(flbase)[0]
        images[flbase_noext]=fl
        images_by_folder[str(train_path)].append(flbase)

In [3]:
path_change_map = {}

for key in list(images_by_folder.keys()):
    temp = key.replace('data_svs', 'data_lowres_1100x1100')
    path_change_map[key] = temp

In [5]:
#image_by_folder.keys()
#images_by_folder
#path_change_map
#op_slide_img = open_slide('/scratch/kk4ze/data_svs/train/EE/17.svs')

In [6]:
#op_slide_img.dimensions

(7488, 9714)

In [7]:
#y_break=math.ceil(op_slide_img.dimensions[1]/op_slide_img.dimensions[0])
#y_break

2

In [56]:
# for count in range(0,y_break):
#     x=op_slide_img.dimensions[0]
#     if count==0:
#         y=0
#     else:    
#         y=0+op_slide_img.dimensions[1]-(count*x)
#     img = op_slide_img.read_region((0,y), 0, (x,x))
#     jpg_img = img.convert('RGB')
#     jpg_img.save('test_'+str(count)+'.jpg')

In [ ]:
width_height = (1100,1100)
for key in images_by_folder.keys():
    for value in images_by_folder[key]:
        img_path = str(key) +'/'+ str(value)
        op_slide_img = open_slide(img_path)
        if op_slide_img.dimensions[0]>20000 or op_slide_img.dimensions[1]>20000:
                continue
        value_strip=value[:-4]
        if op_slide_img.dimensions[1]>op_slide_img.dimensions[0]:
            y_break=math.ceil(op_slide_img.dimensions[1]/op_slide_img.dimensions[0])
            for count in range(0,y_break):
                x=op_slide_img.dimensions[0]
                if count==0:
                    y=0
                else:
                    y=0+op_slide_img.dimensions[1]-(count*x)
                img = op_slide_img.read_region((0,y), 0, (x,x))
                img = img.resize(width_height, Image.ANTIALIAS)
                jpg_img = img.convert('RGB')
                jpg_img.save(path_change_map[key] + '/' + str(value_strip) + '_' + str(op_slide_img.dimensions[0])+'_'+str(op_slide_img.dimensions[1])+'_vert_'+'_'+str(count)+'.jpg')
        elif op_slide_img.dimensions[0]>op_slide_img.dimensions[1]:
            x_break=math.ceil(op_slide_img.dimensions[0]/op_slide_img.dimensions[1])
            for count in range(0,x_break):
                y=op_slide_img.dimensions[1]
                if count==0:
                    x=0
                else:
                    x=0+op_slide_img.dimensions[0]-(count*y)
                img = op_slide_img.read_region((x,0), 0, (y,y))
                img = img.resize(width_height, Image.ANTIALIAS)
                jpg_img = img.convert('RGB')
                jpg_img.save(path_change_map[key] + '/' + str(value_strip) + '_'+ str(op_slide_img.dimensions[0])+'_'+str(op_slide_img.dimensions[1])+'_horiz_'+'_'+str(count)+'.jpg')
        print(key + ' Saved!\n')
        